In [ ]:
import os
from tqdm import tqdm

In [ ]:
from avapi.carla import CarlaScenesManager


# cpath = os.path.join("../data/multi-agent-v1/")
cpath = "/data/shared/CARLA/multi-agent-v1"
# cpath = "../examples/sim_results"
CSM = CarlaScenesManager(cpath)
print(CSM.scenes)
CDM = CSM.get_scene_dataset_by_index(0)
print(f"{len(CDM)} frames")

In [ ]:
# Here we need OpenCOOD. Thankfully, most libraries like pytorch are actually compatible.
# (1) git submodule add -b carla_demo https://github.com/zqzqz/OpenCOOD.git submodules/OpenCOOD (already added in .gitmodules)
# (2) pip install cumm spconv-cu113
# (3) cd submodules/OpenCOOD && python opencood/utils/setup.py build_ext --inplace
# (4) cd submodules/OpenCOOD && python setup.py develop
# (5) Download https://drive.google.com/file/d/11pG0kf2uR9N_o_ACBi_zfd7flGCgZt40/view?usp=sharing into models/
# Folder structure is like models/pointpillar_attentive_fusion/{config.yaml and latest.pth}

import os
import numpy as np
import torch

import opencood.hypes_yaml.yaml_utils as yaml_utils
from opencood.tools import train_utils, inference_utils
from opencood.data_utils.datasets import build_dataset
from opencood.utils import box_utils


class OpencoodPerception:
    def __init__(
        self,
        fusion_method="early",
        model_name="pointpillar",
        model_path="/data/shared/models/opencood/",
    ):
        assert model_name in ["pixor", "voxelnet", "second", "pointpillar"]
        assert fusion_method in ["early", "intermediate", "late"]
        self.name = "{}_{}".format(model_name, fusion_method)
        self.devices = "cuda:0"
        self.model_name = model_name
        self.fusion_method = fusion_method
        self.model_dir = os.path.join(
            model_path,
            "{}_{}_fusion".format(
                self.model_name,
                self.fusion_method
                if self.fusion_method != "intermediate"
                else "attentive",
            ),
        )
        self.config_file = os.path.join(self.model_dir, "config.yaml")

        hypes = yaml_utils.load_yaml(self.config_file, None)
        self.model = train_utils.create_model(hypes)
        self.dataset = build_dataset(hypes, visualize=False, train=False)
        # we assume gpu is necessary
        if torch.cuda.is_available():
            self.model.cuda()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        _, self.model = train_utils.load_saved_model(self.model_dir, self.model)
        self.model.eval()

        self.preprocessors = {
            "early": self.dataset.preprocess_from_carla,
            "intermediate": self.dataset.preprocess_from_carla,
            "late": self.dataset.preprocess_from_carla,
        }
        self.inference_processors = {
            "early": inference_utils.inference_early_fusion,
            "intermediate": inference_utils.inference_intermediate_fusion,
            "late": inference_utils.inference_late_fusion,
        }

    def run(self, multi_vehicle_case, ego_id):
        batch = self.preprocessors[self.fusion_method](multi_vehicle_case, ego_id)
        batch_data = self.dataset.collate_batch_test([batch])
        with torch.no_grad():
            batch_data = train_utils.to_device(batch_data, self.device)
            pred_box_tensor, pred_score, gt_box_tensor = self.inference_processors[
                self.fusion_method
            ](batch_data, self.model, self.dataset)
        if pred_box_tensor is None:
            pred_bboxes = np.array([])
            pred_scores = np.array([])
        else:
            pred_bboxes = pred_box_tensor.cpu().numpy()
            pred_bboxes = box_utils.corner_to_center(pred_bboxes, order="lwh")
            pred_bboxes[:, 2] -= 0.5 * pred_bboxes[:, 5]
            pred_scores = pred_score.cpu().numpy()
        return pred_bboxes, pred_scores


perception = OpencoodPerception(model_name="pointpillar", fusion_method="intermediate")

In [ ]:
def rotation_matrix(roll, yaw, pitch):
    R = np.array(
        [
            [
                np.cos(yaw) * np.cos(pitch),
                np.cos(yaw) * np.sin(pitch) * np.sin(roll) - np.sin(yaw) * np.cos(roll),
                np.cos(yaw) * np.sin(pitch) * np.cos(roll) + np.sin(yaw) * np.sin(roll),
            ],
            [
                np.sin(yaw) * np.cos(pitch),
                np.sin(yaw) * np.sin(pitch) * np.sin(roll) + np.cos(yaw) * np.cos(roll),
                np.sin(yaw) * np.sin(pitch) * np.cos(roll) - np.cos(yaw) * np.sin(roll),
            ],
            [
                -np.sin(pitch),
                np.cos(pitch) * np.sin(roll),
                np.cos(pitch) * np.cos(roll),
            ],
        ]
    )
    return R

## Test Different Perception/Tracking Approaches

In [ ]:
from avstack.geometry import Sphere, Circle, Box3D, Position, Attitude

from avstack.modules.perception.detections import BoxDetection
from avstack.modules.perception.object2dfv import MMDetObjectDetector2D
from avstack.modules.perception.object3d import (
    Passthrough3DObjectDetector,
    MMDetObjectDetector3D,
)
from avstack.modules.tracking.tracker3d import BasicBoxTracker3D
from avstack.modules.tracking.multisensor import MeasurementBasedMultiTracker
from avstack.geometry import GlobalOrigin3D, transform_orientation

from avstack.datastructs import DataContainer

# init models
agents = [0, 1]
percep = Passthrough3DObjectDetector()
# percep = MMDetObjectDetector3D(model="pointpillars", dataset="carla-joint")
trackers = {agent: BasicBoxTracker3D() for agent in agents}
trackers["central"] = BasicBoxTracker3D()

# run loop
dets = {}
tracks = {}
imgs_all = {agent: [] for agent in agents}
pcs_all = {agent: [] for agent in agents}
dets_all = {agent: [] for agent in agents}
dets_all["central"] = []
tracks_all = {agent: [] for agent in agents}
tracks_all["central"] = []
agent_0_frames = CDM.get_frames(sensor="lidar-0", agent=0)[1:-1]
# Just choose 100 frames for testing
agent_0_frames = agent_0_frames

# Use the ego vehicle as center, do collaborative perception.
# The predicted bboxes are also in the ego vehicle's coordination.
ego_agent = agents[0]

for frame in tqdm(agent_0_frames):
    # run perception and individual trackers
    found_data = False
    fovs = {}
    platforms = {}
    perception_input = {}
    for agent in agents:
        ###############################################
        # GET DATA
        ###############################################
        lidar_sensor = "lidar-0"
        camera_sensor = "camera-0"
        img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
        pc = CDM.get_lidar(frame=frame, sensor=lidar_sensor, agent=agent)
        imgs_all[agent].append(img)
        pcs_all[agent].append(pc)
        objs = CDM.get_objects(frame=frame, sensor=lidar_sensor, agent=agent)
        calib = CDM.get_calibration(frame=frame, sensor=lidar_sensor, agent=agent)
        fovs[agent] = Sphere(radius=100)
        platforms[agent] = calib.reference

        # Get LiDAR pose.
        ref_global = pc.calibration.reference.integrate(start_at=GlobalOrigin3D)
        position = ref_global.x
        attitude = ref_global.q
        euler = transform_orientation(attitude, "quat", "euler")
        lidar_pose = np.asarray(
            [
                *position,
                np.degrees(euler[0]),
                np.degrees(euler[2]),
                np.degrees(euler[1]),
            ]
        )
        # lidar_pose = np.zeros(6)

        # ###############################################
        # # DISTRIBUTED PERCEPTION
        # ###############################################
        # found_data = True
        # dets[agent] = percep(objs)
        # dets_all[agent].append(dets[agent])

        # ###############################################
        # # DISTRIBUTED TRACKING USING DISTRIBUTED PERCEP
        # ###############################################
        # tracks[agent] = trackers[agent](dets[agent], platform=calib.reference)
        # if not isinstance(tracks[agent], DataContainer):
        #     raise
        # tracks_all[agent].append([track.box3d.copy() for track in tracks[agent]])

        # Construct the input data structure for collaborative perception model.
        perception_input[agent] = {
            "lidar": pc.data.x,
            # This pose is in OPV2V format (x, y, z, roll, yaw, pitch) and the rotation are in degrees not radians.
            "lidar_pose": lidar_pose,
        }

    ###############################################
    # CENTRALIZED PERCEPTION
    ###############################################
    # The predicted bboxes are in the ego vehicle's coordination.
    # Please transform boxes to global or other vehicle' coordination according to demands.
    pred_bboxes, _ = perception.run(perception_input, ego_agent)

    central_dets = []
    for pred_bbox in pred_bboxes:
        det = BoxDetection(
            "lidar-0",
            Box3D(
                position=Position(x=pred_bbox[0:3], reference=platforms[ego_agent]),
                # Note that Carla coordination has a reverted y axis (?). Taking a negation of yaw seems make it right.
                attitude=Attitude(
                    q=rotation_matrix(0, -pred_bbox[6], 0),
                    reference=platforms[ego_agent],
                ),
                hwl=pred_bbox[3:6][::-1],
            ),
            platforms[ego_agent],
        )
        central_dets.append(det)

    dets_all["central"].append(central_dets)

    # ###############################################
    # # CENTRALIZED TRACKING USING DISTRIBUTED PERCEP
    # ###############################################
    # # run central tracker on all detections
    # if found_data:
    #     tracks["central"] = trackers["central"](
    #         detections=dets,
    #         fovs=fovs,
    #         platforms=platforms,
    #     )
    #     tracks_all["central"].append(tracks["central"])

#### Verify that the union of the two is similar to the central

In [ ]:
# from avapi.visualize.snapshot import show_lidar_bev_with_boxes, show_boxes_bev

# from avstack.geometry import GlobalOrigin3D

# tracks_all_this_frame = tracks[0]+tracks[1]
# tracks_all_this_frame = [track.change_reference(GlobalOrigin3D, inplace=False) for track in tracks_all_this_frame]
# show_boxes_bev(boxes=tracks_all_this_frame)
# show_boxes_bev(boxes=tracks["central"])

# extent = [[-70, 70], [-70, 70], [-100, 100]]
# show_lidar_bev_with_boxes(pc=pc, boxes=tracks_all_this_frame, extent=extent)
# show_lidar_bev_with_boxes(pc=pc, boxes=tracks["central"], extent=extent)

In [ ]:
from avapi.visualize.movie import make_movie


vid_folder = "videos"
os.makedirs(vid_folder, exist_ok=True)


def make_movies(imgs, pcs, dets, tracks, agent, extent=None):

    ###############################################
    # CAMERA-BASED VISUALIZATION
    ###############################################

    # perception movie
    make_movie(
        raw_imgs=imgs,
        raw_pcs=pcs,
        boxes=dets,
        name=os.path.join(vid_folder, f"agent-{agent}-perception-img"),
        save=True,
        show_in_notebook=False,
        projection="img",
        extent=extent,
    )

    # tracking movie
    make_movie(
        raw_imgs=imgs,
        raw_pcs=pcs,
        boxes=tracks,
        name=os.path.join(vid_folder, f"agent-{agent}-tracking-img"),
        save=True,
        show_in_notebook=False,
        projection="img",
        extent=extent,
    )

    ###############################################
    # BEV-BASED VISUALIZATION
    ###############################################

    # perception movie
    make_movie(
        raw_imgs=imgs,
        raw_pcs=pcs,
        boxes=dets,
        name=os.path.join(vid_folder, f"agent-{agent}-perception-bev"),
        save=True,
        show_in_notebook=False,
        projection="bev",
        extent=extent,
    )

    # tracking movie
    make_movie(
        raw_imgs=imgs,
        raw_pcs=pcs,
        boxes=tracks,
        name=os.path.join(vid_folder, f"agent-{agent}-tracking-bev"),
        save=True,
        show_in_notebook=False,
        projection="bev",
        extent=extent,
    )


extent = [[-70, 70], [-70, 70], [-100, 100]]
# for agent in agents:
#     make_movies(
#         imgs=imgs_all[agent],
#         pcs=pcs_all[agent],
#         dets=dets_all[agent],
#         tracks=tracks_all[agent],
#         agent=agent,
#         extent=extent,
#     )

make_movie(
    raw_imgs=imgs_all[ego_agent],
    raw_pcs=pcs_all[ego_agent],
    boxes=dets_all["central"],
    name=os.path.join(vid_folder, f"agent-{ego_agent}-perception-bev"),
    save=True,
    show_in_notebook=False,
    projection="bev",
    extent=extent,
)